In [ ]:
from collections import defaultdict
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
from os.path import join, basename, isdir
from os import listdir, makedirs
import shutil as sh
import pickle

In [ ]:
plt.rcParams['figure.figsize'] = 9, 6

# Sort Trials

In [ ]:
HOME = os.environ["HOME"]
print(HOME)
os.chdir(join(HOME, 'tmp/pretrained-runs/'))

In [ ]:
def get_contents(d, dtype=str):
    with open(d, "r") as f:
        return dtype(f.readline())

def get_source_info(d):
    dsource = get_contents(join(d, "weights-source"), str)
    dsource = dsource.replace("/root/trained-models", join(HOME, "tmp/final-runs"))
    dsource += d[-5:]
    duration = get_contents(join(dsource, "duration"), float)
    gain = get_contents(join(d, "tx-gain"), float)
    mode = "nc" if "classic" in dsource else "nn"
    shared = "shared" if "shared" in dsource else "private"
    return duration, gain, mode, shared

In [ ]:
get_source_info("pretrained-neural-neural-1655856184-srn2")

In [ ]:
dirs = list(filter(lambda d: isdir(d), listdir('.')))
dirs = list(filter(lambda d: 'neural' in d, dirs))
durations = {}
durations['private'] = {'nc': defaultdict(list), 'nn': defaultdict(list)}
durations['shared'] = {'nc': defaultdict(list), 'nn': defaultdict(list)}
for d in dirs:
    t, gain, agents, preamble = get_source_info(d)
    durations[preamble][agents][(t, gain)].append(d)
for k in durations["shared"]["nc"]:
    durations["shared"]["nc"][k].sort()
for k in durations["shared"]["nn"]:
    durations["shared"]["nn"][k].sort()
for k in durations["private"]["nc"]:
    durations["private"]["nc"][k].sort()
for k in durations["private"]["nn"]:
    durations["private"]["nn"][k].sort()
# print(durations)
# Remove shorter tests
keys = list(durations["shared"]["nc"].keys())
for k in keys:
    if k[0] < 120:
        durations["shared"]["nc"].pop(k)
keys = list(durations["shared"]["nn"].keys())
for k in keys:
    if k[0] < 300:
        durations["shared"]["nn"].pop(k)

# Trial Outcomes

In [ ]:
def success_rate(ms): 
    if ms.size <= 40:
        return sum(ms < 0.1) / 40.
    else:
        return sum(ms < 0.1) / 66.

def do_runtime(dirs, sharedstr, agentstr, timestr):
    mins = []
    means = []
    maxes = []
    for d in dirs:
        try:
            with open(join(d, 'results'), 'r') as f:
                lcount, lmean, lmin, lmax, _ = f.readlines()
                mins.append(float(lmin.strip().split()[-1]))
                means.append(float(lmean.strip().split()[-1]))
                maxes.append(float(lmax.strip().split()[-1]))
        except:
            pass

    plt.stem(means)
    plt.yscale('log')
    plt.title("{} preamble {}, {} seconds trials".format(
                sharedstr, agentstr, timestr))
    plt.show()
    print("Success rate", success_rate(np.array(means)))
    return mins, means, maxes

for sharedstr in ("shared", "private"):
    for agentstr in ("nn", "nc"):
        print(' '.join(["=====", sharedstr.upper(), 
                       agentstr.upper(), "====="]))
        for time in durations[sharedstr][agentstr]:
            do_runtime(durations[sharedstr][agentstr][time],
                       sharedstr, agentstr, time)

In [ ]:
print(durations['private']['nn'][(600, 16)])

# SNR vs BER

In [ ]:
def success_rate(ms): 
    if ms.size <= 40:
        return sum(ms < 0.1) / 40.
    else:
        return sum(ms < 0.1) / 66.

def do_snr_ber(dirs, sharedstr, agentstr):
    gains = []
    ber10 = []
    ber50 = []
    ber90 = []
#     baseline = 10. ** -np.arange(1, 6) 
    baseline = [1.04085880e-01, 9.88240808e-03, 7.21992638e-04, 9.59187110e-05, 9.10228289e-06]
    for t, g in dirs:
        gains.append(g)
        mins = []
        means = []
        maxes = []
        for d in dirs[(t,g)]:
            try:
                with open(join(d, 'results'), 'r') as f:
                    lcount, lmean, lmin, lmax, _ = f.readlines()
                    mins.append(float(lmin.strip().split()[-1]))
                    means.append(float(lmean.strip().split()[-1]))
                    maxes.append(float(lmax.strip().split()[-1]))
            except:
                pass
        N = len(means)
        if g > 13: N = N // 2
        n10 = int(np.ceil(N * 0.1))
        n50 = int(np.ceil(N * 0.5))
        n90 = int(np.ceil(N * 0.9))
        ber10.append(sorted(means)[n10])
        ber50.append(sorted(means)[n50])
        ber90.append(sorted(means)[n90])
    # Convert to ndarrays and sort
    gains = np.array(gains)
    ber10 = np.array(ber10)
    ber50 = np.array(ber50)
    ber90 = np.array(ber90)
    isort = np.argsort(gains)
    gains = gains[isort]
    ber10 = ber10[isort]
    ber50 = ber50[isort]
    ber90 = ber90[isort]
    plt.plot(gains, baseline, label="Baseline", marker='o')
    plt.plot(gains, ber10, label="10%", marker='o')
    plt.plot(gains, ber50, label="50%", marker='o')
    plt.plot(gains, ber90, label="90%", marker='o')
    plt.title("{} preamble {}, BER vs Gain(TODO SNR)".format(
                sharedstr, agentstr))
    plt.legend()
    plt.yscale("log")
    plt.show()
    return gains, ber10, ber50, ber90

for sharedstr in ("shared", "private"):
    for agentstr in ("nn", "nc"):
        print(' '.join(["=====", sharedstr.upper(), 
                       agentstr.upper(), "====="]))
        gains, ber10, ber50, ber90 = do_snr_ber(durations[sharedstr][agentstr],
                   sharedstr, agentstr)
        out = {'simulation_equivalent_snr': [4.2, 8.4, 10.4, 12, 13, 15],
               'empirical_snr': [6.52637972, 10.56335866, 12.80612474, 14.15178439, 15.49744404],
               'empirical_baseline': [1.04085880e-01, 9.88240808e-03, 7.21992638e-04, 9.59187110e-05, 9.10228289e-06],
               '10th-percentile': list(ber10),
               '50th-percentile': list(ber50),
               '90th-percentile': list(ber90),
              }
        with open("{}-{}-ber_curve.pkl".format(sharedstr, agentstr), "wb") as f:
            pickle.dump(out, f)